In [ ]:
import json
import pandas as pd
import glob as glob

methods = {
    "Baseline": "qwen-2.5-x.5B-instruct-countdown-baseline",
    "OP": "qwen-2.5-x.5B-instruct-sft-lora-countdown-optimal-seq8k-5k",
    "SoS": "qwen-2.5-x.5B-instruct-sft-lora-countdown-search-seq8k-5k", 
    "RSoS": "qwen-2.5-x.5B-instruct-sft-lora-countdown-search-react-correct-seq10k-5k", 
    "Distill": "qwen-2.5-x.5B-instruct-sft-lora-countdown-deepseek-correct-seq8k-5k"
}

tasks = {
    "Countdown": "test_128", 
    "Countdown-3": "countdown_3num_128", 
    "Countdown-5": "countdown_5num_128",
    "KnK": "knk"
}

sizes = ["0.5B", "1.5B"]

def parse_results_from_json(file):
    try:
        with open(file, 'r') as f:
            data = json.load(f)
        # print("File:", file)
        # Extract the relevant information from the JSON data   
        if "knk" in file:
            return data["scores"]["2ppl"]    
        if "countdown" in file or "test" in file:
            # hyperparams = data[0]['hyperparams']
            return data[1]['mean']*100
    except Exception as e:
        print("Error reading file:", file)
        print("Error message:", e)
        return None

# initialize results to store for sizes, method_key, task_key
results = {size: {method_key: {task_key: None for task_key in tasks.keys()} for method_key in methods.keys()} for size in sizes}

for size_val in sizes:
    for method_key, method_val in methods.items():
        for task_key, task_val in tasks.items():
            folder_to_look_for = f"./{method_val}/{task_val}*.json".replace("x.5B", size_val) 
            specific_file = glob.glob(folder_to_look_for)
            if specific_file:
                results[size_val][method_key][task_key] = parse_results_from_json(specific_file[0])
            
# above works, put their filepath in a pandas df
                 
# put results in a pandas df
df = pd.DataFrame.from_dict({(i, j): results[i][j] 
                           for i in results.keys() 
                           for j in results[i].keys()},
                           orient='index')
# Tranpose
df = df.transpose()
# make the df float up to 2 decimals
df = df.round(2)
df.to_latex("results.tex", index=True, float_format="%.2f")
df

0.5B                                1.5B                      \
            Baseline    OP    SoS  RSoS Distill Baseline    OP    SoS   RSoS   
Countdown        0.0  2.34  37.50  0.78   25.78     0.78  5.47  49.22  52.34   
Countdown-3      0.0  1.56  33.59  0.00   42.97     3.12  4.69  57.81  62.50   
Countdown-5      0.0  0.78   0.00  0.00    0.78     0.00  0.78   0.00   0.00   
KnK              1.0  1.50   0.00  0.00    1.50     7.00  2.50   0.00   0.00   

                     
            Distill  
Countdown     44.53  
Countdown-3   71.09  
Countdown-5    1.56  
KnK           12.00

In [10]:
sizes = ["1.5B"]

methods = {
    "No rejection \nsampling": "qwen-2.5-x.5B-instruct-sft-lora-countdown-search-react-seq10k-5k",
    "With rejection \nsampling": "qwen-2.5-x.5B-instruct-sft-lora-countdown-search-react-correct-seq10k-5k"
}

tasks = {
    "Countdown:": "test_128", 
}

results = {size: {method_key: {task_key: None for task_key in tasks.keys()} for method_key in methods.keys()} for size in sizes}
for size_val in sizes:
    for method_key, method_val in methods.items():
        for task_key, task_val in tasks.items():
            folder_to_look_for = f"./{method_val}/{task_val}*.json".replace("x.5B", size_val) 
            specific_file = glob.glob(folder_to_look_for)
            if specific_file:
                results[size_val][method_key][task_key] = parse_results_from_json(specific_file[0])
            
# above works, put their filepath in a pandas df
                
# put results in a pandas df
df = pd.DataFrame.from_dict({(i, j): results[i][j] 
                        for i in results.keys() 
                        for j in results[i].keys()},
                        orient='index')
# Tranpose
df = df.transpose()
# make the df float up to 2 decimals
df = df.round(2)
df.to_latex("results.tex", index=True, float_format="%.2f")
df

1.5B                          
           No rejection \nsampling With rejection \nsampling
Countdown:                     NaN                     52.34

In [19]:
results = {size: {method_key: {task_key: None for task_key in tasks.keys()} for method_key in methods.keys()} for size in training_sizes}
results

{'1k': {'SoS': {'Countdown:': None}, 'Distill': {'Countdown:': None}},
 '5k': {'SoS': {'Countdown:': None}, 'Distill': {'Countdown:': None}}}

In [21]:
training_sizes = ["1k", "5k"]

methods = {
    "SoS": "qwen-2.5-1.5B-instruct-sft-lora-countdown-search",
    "Distill": "qwen-2.5-1.5B-instruct-sft-lora-countdown-deepseek-correct-seq8k"
}

tasks = {
    "Countdown:": "test_128", 
}

results = {size: {method_key: {task_key: None for task_key in tasks.keys()} for method_key in methods.keys()} for size in training_sizes}
for size_val in training_sizes:
    for method_key, method_val in methods.items():
        for task_key, task_val in tasks.items():
            folder_to_look_for = f"./{method_val}*{size_val}/{task_val}*.json"
            specific_file = glob.glob(folder_to_look_for)
            if specific_file:
                results[size_val][method_key][task_key] = parse_results_from_json(specific_file[0])
            
# above works, put their filepath in a pandas df
                
# put results in a pandas df
df = pd.DataFrame.from_dict({(i, j): results[i][j] 
                        for i in results.keys() 
                        for j in results[i].keys()},
                        orient='index')
# Tranpose
df = df.transpose()
# make the df float up to 2 decimals
df = df.round(2)
df.to_latex("1k-vs-5k.tex", index=True, float_format="%.2f")
df

1k             5k        
              SoS Distill    SoS Distill
Countdown:  32.81   31.25  52.34   44.53

In [41]:
# load json
with open("/cs/student/msc/ml/2024/ycheah/projects/sos/stream-of-search/results/qwen-2.5-1.5B-instruct-sft-lora-countdown-deepseek-correct-seq8k-5k/test_128_20250406-195241.json", 'r') as f:
    data = json.load(f)
    
i = 22
print(f"Solved: {data[i]['parsed_results']['solved']} \n", data[i]['completion'])

Solved: False 
 system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
user
Combine these initial numbers [59, 1, 92, 34] using only arithmetic operations (+, -, *, /) to reach the target value 53. All initial numbers must be used exactly once.
Conclude with the final result in EXACTLY this format:
```
SOLUTION: YES/NO
OPERATIONS: list of string of operations performed, each string involving only 1 operation. For example, ['A+B=C','C+D=E'] is allowed, ['A+B+D=E'] is not allowed
RESULT: final_value
```

Note that the solution does exist. Verify your solutions before your present your final results and backtrack to correct mistakes from before your mistakes if you have to.

assistant
Okay, let's see. I need to combine the numbers 59, 1, 92, and 34 using each exactly once with arithmetic operations to get 53. Hmm. Let me start by thinking about possible combinations.

First, maybe subtract some big numbers? Like 92 minus something. 92 - 59 is 33. Then we have 33 plus 

In [74]:
sizes = ["0.5B", "1.5B"]

methods = {
    "Without verification": "qwen-2.5-x.5B-instruct-sft-lora-countdown-deepseek-correct-seq8k-5k/no-backtrack",
    "With verification": "qwen-2.5-x.5B-instruct-sft-lora-countdown-deepseek-correct-seq8k-5k"
}

tasks = {
    "Countdown": "test_128", 
    "Countdown-3": "countdown_3num_128", 
    "Countdown-5": "countdown_5num_128",
    "KnK": "knk"
}

results = {size: {method_key: {task_key: None for task_key in tasks.keys()} for method_key in methods.keys()} for size in sizes}
for size_val in sizes:
    for method_key, method_val in methods.items():
        for task_key, task_val in tasks.items():
            folder_to_look_for = f"./{method_val}/{task_val}*.json".replace("x.5B", size_val) 
            specific_file = glob.glob(folder_to_look_for)
            if specific_file:
                results[size_val][method_key][task_key] = parse_results_from_json(specific_file[0])
            
# put results in a pandas df
df = pd.DataFrame.from_dict({(i, j): results[i][j] 
                        for i in results.keys() 
                        for j in results[i].keys()},
                        orient='index')

# Create teacher_df with same structure as df (using MultiIndex)
teacher_results = {
    "Countdown": {"Without verification": 68.0, "With verification": 78.0},
    "Countdown-3": {}, 
    "Countdown-5": {},
    "KnK": {}
}

# Create teacher_df with proper structure - (model, method) as index, tasks as columns
teacher_data = {}
for method in ["Without verification", "With verification"]:
    teacher_data[("Teacher", method)] = {task: teacher_results[task].get(method, pd.NA) for task in tasks.keys()}

teacher_df = pd.DataFrame.from_dict(teacher_data, orient='index')

# Now both df and teacher_df have the same structure - combine them
combined_df = pd.concat([df, teacher_df])

# Reorder to put Teacher results first
model_order = ["Teacher", "0.5B", "1.5B"]
method_order = ["Without verification", "With verification"]
idx = pd.MultiIndex.from_product([model_order, method_order])
combined_df = combined_df.reindex(idx)

# Clean up - remove rows with no data
combined_df = combined_df.dropna(how='all')

# make the df float up to 2 decimals
combined_df = combined_df.round(2)
combined_df.to_latex("backtracking.tex", index=True, float_format="%.2f")
combined_df

/tmp/ipykernel_546305/2749351876.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([df, teacher_df])


Countdown  Countdown-3  Countdown-5   KnK
Teacher Without verification      68.00          NaN          NaN   NaN
        With verification         78.00          NaN          NaN   NaN
0.5B    Without verification      21.09        35.94         0.00   0.0
        With verification         25.78        42.97         0.78   1.5
1.5B    Without verification      37.50        71.09         0.78  10.0
        With verification         44.53        71.09         1.56  12.0